# Filtering Patents by Category 'C'

We filter patent, First for each big XML file, we split the patents, then we only need Chimestry Patents, so we look for Category C under <section> element, then write each patent in a separate XML file.

In [ ]:
import xml.etree.ElementTree as ET

# Read the XML file
with open("Data/ipg150616.xml", "r") as file:
    xml_content = file.read()

# Split the XML file into separate documents
xml_documents = xml_content.split("<?xml version=")

# Remove the empty first element
xml_documents = xml_documents[1:]

# Parse each XML document individually
for i, xml_doc in enumerate(xml_documents):
    # Add back the XML declaration
    xml_doc = "<?xml version=" + xml_doc
    
    # Parse the XML document
    root = ET.fromstring(xml_doc)
    closels_lvl = root.findall(".//section")
    if closels_lvl and closels_lvl[0].text.lower().strip()=='c':
        print(closels_lvl[0].text)
        file1 = open(f"Data/ipg150616/ipg150616_{closels_lvl[0].text}_{i}.xml","w")
        file1.write(xml_doc)
        file1.close()

# Transforming XML to TXT

From Target patents, we extract text under the elements <description>, <claims>, <abstract>

In [1]:
import xml.etree.ElementTree as ET
import re
import string

def extract_text_from_xml(xml_path):
    # Parse the XML file
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Define the namespace used in the XML file
    namespace = {
        "us-patent-grant": "http://www.w3.org/2005/Atom",
        "us-bibliographic-data-grant": "http://www.w3.org/2005/Atom",
        # Add other namespaces if necessary
    }

    # Extract all text from the XML
    all_text = []
    elements  = root.findall("description")
    elements.extend(root.findall("claims"))
    elements.extend(root.findall('abstract'))

    def extract_text(element):
        if element.text:
            all_text.append(element.text)
        for child in element:
            extract_text(child)

    for element in elements:
        extract_text(element)

    # Join the extracted text into a single string
    extracted_text = " ".join(all_text)
    
    return preprocess_text(extracted_text)
  

def preprocess_text(text):
    # Remove repeated characters
    text = re.sub(r'(.)\1+', r'\1', text)
    
    # Remove non-English characters
    text = ''.join(char for char in text if char in string.printable and char.isascii())
    text = text.replace(r"\n+", "\n" )
    text = text.replace(r"\s+", "\s" )
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
    return text.lower()

## Extractin text from first 20 Patents then write it to TXT file for later usages

In [2]:
import os

directory = 'Data/ipa221229/'  # Replace with the path to your directory

# Get the list of files in the directory
file_list = os.listdir(directory)[10:15]


In [3]:
extract_text_from_xml("Data/ipa221229/ipa221229_C_3484.xml")

text = []

for file in file_list:
    file_name = f"Data/ipa221229/{file}"
    
    text.append(extract_text_from_xml(file_name))
    

In [4]:
import html

with open('Data/text_data_ipa221229_5_patents.txt', 'w', encoding='utf-8') as f:
    for line in text:
        f.write(html.unescape(line))
        f.write('\n')